<a href="https://colab.research.google.com/github/UDDSI/COVID19hackathon/blob/master/projects/SupportTrack/how_to_geocode_addresse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# author: Federica Bianco (@fedhere)

geocoding example


In [34]:
# read in some data
import pandas as pd
import numpy as np
exampledata = pd.read_csv("https://raw.githubusercontent.com/UDDSI/COVID19hackathon/master/projects/SupportTrack/testaddresses.csv")
exampledata

,Unnamed: 0,Meeting Name,Location
0,1,Serenity at Sunrise,St Thomas Episcopal Church
1,2,Pioneer Group,St. Andrew's Church
2,3,Dawn Patrol,Westminster House - First Presbyterian Church
3,4,Early Risers,Trinity Episcopal Parish
4,5,Sunrise,Twelve and Twelve Club
5,6,Eye Opener,Red Clay Creek Presbyterian Churchi
6,7,New Day,St Nicholas Episcopal Church
7,8,Midday Miracles,First Baptist Church Of Kennett Square
8,9,First Stop Friday,Tower Hill Schooli


In [35]:
#install and runn geopy for geocoding: does not require an api key
!pip install geopy
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter 

#locator is a function with a defined what software you want to use
locator = Nominatim(user_agent='myGeocoder')
# this is the workhorse: the function you will pass to the dataframe series (column) to convert it
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.5)
#location is the address, but it only includes the street or some vague location, we should add at least the state: DE
#because its a string we can just sum it to the other string ",DE"
exampledata["Location"] = exampledata["Location"] + ",DE"
# the apply the geocode to that modified location, this geocode contains proper address and coordinates
exampledata["geocodedloc"] = exampledata["Location"].apply(geocode)
exampledata

,Unnamed: 0,Meeting Name,Location,geocodedloc
0,1,Serenity at Sunrise,"St Thomas Episcopal Church,DE","(Saint Thomas Episcopal Church, Sky Manor Road..."
1,2,Pioneer Group,"St. Andrew's Church,DE","(St. Andrew's Church, Whannels Road, Ward 61, ..."
2,3,Dawn Patrol,"Westminster House - First Presbyterian Church,DE","(First Presbyterian Church, Cedarwood Street, ..."
3,4,Early Risers,"Trinity Episcopal Parish,DE","(Trinity Episcopal Parish, Kimball Avenue, Ora..."
4,5,Sunrise,"Twelve and Twelve Club,DE","(Twelve Twelve, Laurel Street, The Gulch West..."
5,6,Eye Opener,"Red Clay Creek Presbyterian Churchi,DE",None
6,7,New Day,"St Nicholas Episcopal Church,DE","(Saint Nicholas Episcopal Church, Mayflower Dr..."
7,8,Midday Miracles,"First Baptist Church Of Kennett Square,DE","(First Baptist Church of Kennett Square, West ..."
8,9,First Stop Friday,"Tower Hill Schooli,DE",None


In [36]:
# each new entry of the df is a Location object
exampledata["geocodedloc"][0]

Location(Saint Thomas Episcopal Church, Sky Manor Road, Alvater Corner, Franklin Township, Hunterdon County, New Jersey, 08867, United States of America, (40.5606581, -74.9890584, 0.0))

In [37]:
# the first piece of the location object is the address
exampledata["geocodedloc"][0][0]

'Saint Thomas Episcopal Church, Sky Manor Road, Alvater Corner, Franklin Township, Hunterdon County, New Jersey, 08867, United States of America'

In [38]:
# the second piece of the location object is the address
exampledata["geocodedloc"][0][1]

(40.5606581, -74.9890584)

In [39]:
# save the coordinates in a new dataframe column
# the first piece of the location object is the address
exampledata["coords"] = [l[1] for l in exampledata["geocodedloc"]]


TypeError: ignored

In [0]:
#this throws an error because one of the locations was not geocoded: the last row has None at the location place
exampledata["coords"] = [l[1] if not l is None else (np.nan, np.nan) for l in exampledata["geocodedloc"]] 

In [41]:
exampledata


,Unnamed: 0,Meeting Name,Location,geocodedloc,coords
0,1,Serenity at Sunrise,"St Thomas Episcopal Church,DE","(Saint Thomas Episcopal Church, Sky Manor Road...","(40.5606581, -74.9890584)"
1,2,Pioneer Group,"St. Andrew's Church,DE","(St. Andrew's Church, Whannels Road, Ward 61, ...","(13.079534500000001, 80.2640593976282)"
2,3,Dawn Patrol,"Westminster House - First Presbyterian Church,DE","(First Presbyterian Church, Cedarwood Street, ...","(33.7586111, -117.9944444)"
3,4,Early Risers,"Trinity Episcopal Parish,DE","(Trinity Episcopal Parish, Kimball Avenue, Ora...","(42.4508435, -92.3556745)"
4,5,Sunrise,"Twelve and Twelve Club,DE","(Twelve Twelve, Laurel Street, The Gulch West...","(36.15361185, -86.78633588189373)"
5,6,Eye Opener,"Red Clay Creek Presbyterian Churchi,DE",None,"(nan, nan)"
6,7,New Day,"St Nicholas Episcopal Church,DE","(Saint Nicholas Episcopal Church, Mayflower Dr...","(43.5523059, -70.3731052)"
7,8,Midday Miracles,"First Baptist Church Of Kennett Square,DE","(First Baptist Church of Kennett Square, West ...","(39.8463314, -75.71653844057379)"
8,9,First Stop Friday,"Tower Hill Schooli,DE",None,"(nan, nan)"


this is a particularly bad result, only 2 addesses are probably right, that is because the addresses were very vague at the beginning. but it still demonstrated the concept and also the perils of it!